In [ ]:
# verificar que tengan instalado la librería 'pyspark', no requerido en AWS EMR/Spark
!pip install pyspark

In [ ]:
# directorios (path) de entrada y salida:
# 
path_in="../datasets/"
path_out="../out/"
filenametxt_in='in.txt'
filenametxt_out='out.txt'
filenamecsv_in='in.csv'
filenamecsv_out='out.csv'

In [ ]:
#create spark session, no requerido en AWS EMR/Spark
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('nlp').getOrCreate()

In [ ]:
df=spark.createDataFrame([(1,'I really liked this movie'),
                         (2,'I would recommend this movie to my friends'),
                         (3,'movie was alright but acting was horrible'),
                         (4,'I am never watching that movie ever again')],
                        ['user_id','review'])

In [ ]:
df.show(5,False)

In [ ]:
# Tokenization

In [ ]:
from pyspark.ml.feature import Tokenizer

In [ ]:
tokenization=Tokenizer(inputCol='review',outputCol='tokens')

In [ ]:
tokenized_df=tokenization.transform(df)

In [ ]:
tokenized_df.show(4,False)

In [ ]:
# stopwords removal 

In [ ]:
from pyspark.ml.feature import StopWordsRemover

In [ ]:
stopword_removal=StopWordsRemover(inputCol='tokens',outputCol='refined_tokens')

In [ ]:
refined_df=stopword_removal.transform(tokenized_df)

In [ ]:
refined_df.select(['user_id','tokens','refined_tokens']).show(10,False)

In [ ]:
# Movies reviews 

In [ ]:
text_df=spark.read.csv('../datasets/movie_reviews.csv',inferSchema=True,header=True,sep=',')
# datos desde S3
# text_df=spark.read.csv('s3://bucket_name/datasets/movie_reviews.csv',inferSchema=True,header=True,sep=',')

In [ ]:
text_df.printSchema()

In [ ]:
text_df.count()

In [ ]:
# Data Cleaning

In [ ]:
tokenization=Tokenizer(inputCol='Review',outputCol='tokens')

In [ ]:
tokenized_df=tokenization.transform(text_df)

In [ ]:
tokenized_df.show()

In [ ]:
stopword_removal=StopWordsRemover(inputCol='tokens',outputCol='refined_tokens')

In [ ]:
refined_text_df=stopword_removal.transform(tokenized_df)

In [ ]:
refined_text_df.show()

In [ ]:
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import *

In [ ]:
len_udf = udf(lambda s: len(s), IntegerType())

refined_text_df = refined_text_df.withColumn("token_count", len_udf(col('refined_tokens')))


In [ ]:
refined_text_df.orderBy(rand()).show(10)

In [ ]:
refined_text_df.coalesce(1).write.format("csv").option("header", "true").save(path_out+filenamecsv_out)